In [65]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer
import torch.nn as nn
import torch
import torch.optim as optim


# Veri Ön İşleme

In [66]:
# JSON dosyasını okuma
df = pd.read_json("/content/metacriticall.json")

In [67]:
df.head()

,title,metascore,user_score,year,summary,production_company,release_date,duration,rating,genres,url
0,Seven Samurai,98,9.0,1956.0,Seven Samurai (Shichinin no samurai) tells the...,Toho,"Nov 19, 1956",3 h 27 m,Not Rated,"[Action, Drama]",https://www.metacritic.com/movie/seven-samurai...
1,Pinocchio,99,8.2,1940.0,"A living puppet, with the help of a cricket as...",Walt Disney Animation Studios,"Feb 23, 1940",1 h 28 m,Passed,"[Animation, Adventure, Comedy, Family, Fantasy...",https://www.metacritic.com/movie/pinocchio-1940/
2,"4 Months, 3 Weeks and 2 Days",97,8.0,2008.0,"During the final days of communism in Romania,...","Mobra Films,\n Centrul National al Cine...","Jan 23, 2008",1 h 53 m,Not Rated,[Drama],https://www.metacritic.com/movie/4-months-3-we...
3,Psycho,97,8.9,1960.0,"A Phoenix secretary embezzles $40,000 from her...","Alfred J. Hitchcock Productions,\n Sham...","Sep 8, 1960",1 h 49 m,TV-14,"[Horror, Mystery, Thriller]",https://www.metacritic.com/movie/psycho-1960/
4,Ratatouille,96,8.5,2007.0,Despite his sensational sniffer and sophistica...,"Walt Disney Pictures,\n Pixar Animation...","Jun 29, 2007",1 h 51 m,TV-G,"[Animation, Adventure, Comedy, Family, Fantasy]",https://www.metacritic.com/movie/ratatouille/


Şuan da verimizde sütün değerleri genres gibi birden fazla değer içeren sütun bulunduğundan dataframe'e çevirdiğimizde object türünde gözüküyor.

In [68]:
print(df.dtypes)

title                  object
metascore              object
user_score             object
year                  float64
summary                object
production_company     object
release_date           object
duration               object
rating                 object
genres                 object
url                    object
dtype: object


Bazı sütunlarda boş değer bulunuyor.

In [69]:
print(df.isnull().sum())

title                    0
metascore                0
user_score               2
year                   108
summary                  0
production_company     234
release_date           108
duration                 4
rating                1499
genres                   6
url                      0
dtype: int64


İlk olarak sayısal sütunları düzenliyoruz. 'metascore', 'user_score' ve 'year' sütunlarını sayısal veri tipine dönüştürüyoruz.

In [70]:
#Sayısal veri tiplerimizi integer ve float veri tipine dönüştürüyoruz.
df['metascore'] = pd.to_numeric(df['metascore'], errors='coerce')
df['user_score'] = pd.to_numeric(df['user_score'], errors='coerce')                #Sayı formatına uygun olmayanları NaN ile dolduruyoruz.
df['year'] = pd.to_numeric(df['year'], errors='coerce')

Ardından sayısal verilerdeki boş değerler için median ile dolduruyoruz.

In [71]:
#Veri setimizdeki saysısal vri türlerini buluyoruz.
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns

#Sayısal veri setimizdeki eksik değerleri median ile dolduruyoruz.
median_imputers = SimpleImputer(strategy='median')
df[numerical_columns] = median_imputers.fit_transform(df[numerical_columns])

In [72]:
print(df.isnull().sum())

title                    0
metascore                0
user_score               0
year                     0
summary                  0
production_company     234
release_date           108
duration                 4
rating                1499
genres                   6
url                      0
dtype: int64


Object veri türündeki boşluklarımızı boşluk değeri yada unknown değeri atıyoruz. Modelimizin durumuna göre değer atayabiliriz.İstersek en sık kullanılan ifadeler şeklinde de bir atama yapabiliriz ama kullandığım veri seti film veri seti olduğu için genellikle özgün değerlerden oluşuyor.

In [73]:
df['summary'] = df['summary'].fillna('')
df['production_company'] = df['production_company'].fillna('')
df['release_date'] = df['release_date'].fillna('')
df['duration'] = df['duration'].fillna('')
df['rating'] = df['rating'].fillna('unknown')
df['genres']=df['genres'].fillna('')

Genres sütunumuz birden fazla değer içerdiği için liste şeklinde bulunuyor. Bunun için one-hot encoding veya Label encoding yapabiliriz. Genres içerisindeki her türü virgülle ayırarak listeye dönüştürür. Her bir türe "x" adını verdik. Eğer her bir X(tür) string ise split() ile bölme işlemini uygular . Liste halinde ise işlem yapmadan bırakır.

In [74]:
df['genres'] = df['genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Veri Tabanımızı oluşturuyoruz

In [75]:
conn = sqlite3.connect('filmler.db') #Filmler adın da database oluşturuyoruz.
cursor=conn.cursor()                 #Cursor, database üzerinde sql sorgusu yapmamızı sağlar.

# Film bilgilerini içeren tablomuzu oluşturuyoruz.
cursor.execute('''
    CREATE TABLE IF NOT EXISTS filmler (
        title TEXT,
        metascore INTEGER,
        user_score REAL,
        year INTEGER,
        summary TEXT,
        production_company TEXT,
        release_date TEXT,
        duration TEXT,
        rating TEXT,
        genres TEXT,
        url TEXT
    )
''')

# Pandas dataFrame'ine çevirdiğimiz dosyamızı filmler veritabanına tablo olarak aktarıyoruz.
df.to_sql('filmler', conn, if_exists='replace', index=False)

conn.close()

# İşbirlikçi ve İçerik Tabanlı Filtrelemenin Uygulanması

Öneri sistemimizi uygulamak için içerik bazlı filtreleme kullanıyoruz.

Girdi olarak verilen bir film için benzer içerikli filmlerin bulunmasını istiyoruz

In [76]:
#Filmler veritabanımıza bağlanıyoruz. Ve verileri çekiyoruz.
conn = sqlite3.connect('filmler.db')
df = pd.read_sql_query("SELECT * FROM filmler", conn)
conn.close()

In [77]:
df.head()

,title,metascore,user_score,year,summary,production_company,release_date,duration,rating,genres,url
0,Seven Samurai,98.0,9.0,1956.0,Seven Samurai (Shichinin no samurai) tells the...,Toho,"Nov 19, 1956",3 h 27 m,Not Rated,"Action, Drama",https://www.metacritic.com/movie/seven-samurai...
1,Pinocchio,99.0,8.2,1940.0,"A living puppet, with the help of a cricket as...",Walt Disney Animation Studios,"Feb 23, 1940",1 h 28 m,Passed,"Animation, Adventure, Comedy, Family, Fantasy,...",https://www.metacritic.com/movie/pinocchio-1940/
2,"4 Months, 3 Weeks and 2 Days",97.0,8.0,2008.0,"During the final days of communism in Romania,...","Mobra Films,\n Centrul National al Cine...","Jan 23, 2008",1 h 53 m,Not Rated,Drama,https://www.metacritic.com/movie/4-months-3-we...
3,Psycho,97.0,8.9,1960.0,"A Phoenix secretary embezzles $40,000 from her...","Alfred J. Hitchcock Productions,\n Sham...","Sep 8, 1960",1 h 49 m,TV-14,"Horror, Mystery, Thriller",https://www.metacritic.com/movie/psycho-1960/
4,Ratatouille,96.0,8.5,2007.0,Despite his sensational sniffer and sophistica...,"Walt Disney Pictures,\n Pixar Animation...","Jun 29, 2007",1 h 51 m,TV-G,"Animation, Adventure, Comedy, Family, Fantasy",https://www.metacritic.com/movie/ratatouille/


In [78]:
# TF-IDF vektörlerini oluşturma (sadece özet için)
tfidf_summary = TfidfVectorizer(stop_words='english')
tfidf_matrix_summary = tfidf_summary.fit_transform(df['summary']) #Matris oluşturuyoruz.

Önerimizi sunarken kosinüs benzerliği metodunu kullanacağız.

In [79]:
# Kosinüs benzerliğini hesaplama (özet için)
cosine_sim_summary = cosine_similarity(tfidf_matrix_summary, tfidf_matrix_summary) #Burada summary için iki vektör arasındaki açıya bağlı bir skalar değer elde etmiş oluyoruz.

In [80]:
# Önce NaN değerleri boş bir listeye çevirin ve türleri ayırın
df['genres'] = df['genres'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

# MultiLabelBinarizer ile türleri dönüştürme
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(df['genres'])  # Burada df['genres'] doğru formatta olmalı
genres_df = pd.DataFrame(genres_encoded, columns=mlb.classes_, index=df.index)

print(genres_df.head())  # Oluşan genres_df'in doğru olup olmadığını görmek için


      Action  Adventure  Animation  Biography  Comedy  Crime  Documentary  \
0  0       1          0          0          0       0      0            0   
1  0       0          1          1          0       1      0            0   
2  0       0          0          0          0       0      0            0   
3  0       0          0          0          0       0      0            0   
4  0       0          1          1          0       1      0            0   

   Drama  Family  ...  News  Reality-TV  Romance  Sci-Fi  Sport  Talk-Show  \
0      1       0  ...     0           0        0       0      0          0   
1      0       1  ...     0           0        0       0      0          0   
2      1       0  ...     0           0        0       0      0          0   
3      0       0  ...     0           0        0       0      0          0   
4      0       1  ...     0           0        0       0      0          0   

   Thriller  Unknown  War  Western  
0         0        0    0      

In [81]:
# Kosinüs benzerliğini hesaplama (türler için)
cosine_sim_genres = cosine_similarity(genres_df, genres_df)                      #Burada genres için iki vektör arasındaki açıya bağlı bir skalar değer elde etmiş oluyoruz.


In [82]:
# İki benzerlik skorunu birleştirme (örneğin, ortalama alarak)
combined_cosine_sim = (cosine_sim_summary + cosine_sim_genres) / 2

Şimdi bir fonksiyon ile bu benzerliklere göre önerileri getiren bir fonksiyon uyguluyoruz.

In [83]:
 # Film isimlerini 'title' alanına göre indeksleyelim
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [84]:
def reco(title, combined_cosine_sim=combined_cosine_sim, df=df):

    idx = indices[title]

    sim_scores = list(enumerate(combined_cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1][0] if isinstance(x[1], (np.ndarray, list)) else x[1], reverse=True)       #(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    film_indices = [i[0] for i in sim_scores]

    # En benzer filmleri döndürür
    return df['title'].iloc[film_indices]

# NumPy'ı içe aktar
#import numpy as np

oneriler = reco("Harry Potter and the Prisoner of Azkaban")
print("Önerilen Filmler:")
print(oneriler)


Önerilen Filmler:
9354          Harry Potter and the Chamber of Secrets
9925     Harry Potter and the Deathly Hallows: Part I
14838             Harry Potter and the Goblet of Fire
12289       Harry Potter and the Order of the Phoenix
15623    Harry Potter and the Deathly Hallows: Part 2
14193          Harry Potter and the Half-Blood Prince
3711                                     Return to Oz
10198           Harry Potter and the Sorcerer's Stone
6527                              Alice in Wonderland
2578                  Alice Through the Looking Glass
Name: title, dtype: object


# Derin Öğrenme İle Tavsiye Sisteminin İyileştirilmesi


In [85]:
conn = sqlite3.connect('filmler.db')
df = pd.read_sql_query("SELECT * FROM filmler", conn)
conn.close()

In [86]:
# 2. Özet ve Tür Vektörlerinin Oluşturulması
# TF-IDF vektörleri oluşturma (sadece özet için)
tfidf_summary = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix_summary = tfidf_summary.fit_transform(df['summary'])

In [87]:
# Türlerin çok etiketli (multi-label) formatta kodlanması
df['genres'] = df['genres'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(df['genres'])
genres_df = pd.DataFrame(genres_encoded, columns=mlb.classes_, index=df.index)

In [88]:
# TF-IDF matrisini PyTorch tensörüne dönüştür
summary_tensor = torch.tensor(tfidf_matrix_summary.toarray(), dtype=torch.float32)
genres_tensor = torch.tensor(genres_encoded, dtype=torch.float32)

In [89]:
# 3. Model Tanımı
class MovieRecommendationModel(nn.Module):
    def __init__(self, summary_dim, genres_dim, embedding_dim=128):
        super(MovieRecommendationModel, self).__init__()

        # Özet için tam bağlantılı katman
        self.summary_fc = nn.Linear(summary_dim, embedding_dim)
        self.genres_fc = nn.Linear(genres_dim, embedding_dim)

        # Çıkış katmanı
        self.output = nn.Linear(embedding_dim, embedding_dim)
        self.relu = nn.ReLU()

    def forward(self, summary_vector, genres_vector):
        summary_embedding = self.relu(self.summary_fc(summary_vector))
        genres_embedding = self.relu(self.genres_fc(genres_vector))

        # Özellikleri birleştirme
        combined = summary_embedding + genres_embedding
        combined = self.output(combined)

        # Normalizasyon (benzerlik ölçümü için)
        return combined / combined.norm(dim=1, keepdim=True)

In [90]:
# Modeli başlat
summary_dim = summary_tensor.shape[1]
genres_dim = genres_tensor.shape[1]
embedding_dim = 128

In [91]:
model = MovieRecommendationModel(summary_dim, genres_dim, embedding_dim)

In [92]:
# 4. Öneri Yapma Fonksiyonu
def recommend_movies(title, model, df, top_k=10):
    # Belirtilen filmin özet ve tür vektörlerini al
    idx = df[df['title'] == title].index[0]
    summary_vector = summary_tensor[idx]
    genres_vector = genres_tensor[idx]

    # Modelden film vektörlerini al
    with torch.no_grad():
        movie_embeddings = model(summary_tensor, genres_tensor)

    # Seçili film için vektör çıkar
    selected_movie_embedding = movie_embeddings[idx].unsqueeze(0)

    # Kosinüs benzerlik hesaplama
    similarities = torch.mm(selected_movie_embedding, movie_embeddings.T).squeeze(0)
    _, top_indices = torch.topk(similarities, top_k + 1)  # İlk filmi kendisi olarak sayar

    # Önerilen filmleri al
    recommended_movies = df.iloc[top_indices[1:].cpu().numpy()]['title']  # İlk film hariç diğerlerini al
    return recommended_movies

# Örnek bir film için öneriler
movie_title = "Harry Potter and the Prisoner of Azkaban"
recommended_movies = recommend_movies(movie_title, model, df)
print("Önerilen Filmler:")
print(recommended_movies)

Önerilen Filmler:
14838                  Harry Potter and the Goblet of Fire
9354               Harry Potter and the Chamber of Secrets
9925          Harry Potter and the Deathly Hallows: Part I
6527                                   Alice in Wonderland
15623         Harry Potter and the Deathly Hallows: Part 2
3711                                          Return to Oz
2578                       Alice Through the Looking Glass
10198                Harry Potter and the Sorcerer's Stone
13594    The Chronicles of Narnia: The Lion, the Witch ...
3247            The Adventurer: The Curse of the Midas Box
Name: title, dtype: object


# Derin Öğrenme İle Tavsiye Sisteminin İyileştirilmesi- Loos Hesaplanması


In [93]:
# Veri yükleme
conn = sqlite3.connect('filmler.db')
df = pd.read_sql_query("SELECT * FROM filmler", conn)
conn.close()

In [94]:
tfidf_summary=TfidfVectorizer(stop_words='english',max_features=5000)#en fazla beş bin benzersiz kelime vektörü oluşturulur.
tfidf_matrix_summary=tfidf_summary.fit_transform(df['summary'])

In [95]:
df['genres'] = df['genres'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(df['genres'])
genres_df = pd.DataFrame(genres_encoded, columns=mlb.classes_, index=df.index)

In [96]:
#Pytorch kullanabilmemiz için verilerimizi tensor'e çevirmeliyiz.
summary_tensor = torch.tensor(tfidf_matrix_summary.toarray(), dtype=torch.float32)
genres_tensor = torch.tensor(genres_encoded, dtype=torch.float32)

In [97]:
#Model Tanımı
class MovieRecommendationModel(nn.Module):
    def __init__(self, summary_dim, genres_dim, embedding_dim=128):
        super(MovieRecommendationModel, self).__init__()
        self.summary_fc = nn.Linear(summary_dim, embedding_dim)
        self.genres_fc = nn.Linear(genres_dim, embedding_dim)
        self.output = nn.Linear(embedding_dim, embedding_dim)
        self.relu = nn.ReLU()

    def forward(self, summary_vector, genres_vector):
        summary_embedding = self.relu(self.summary_fc(summary_vector))
        genres_embedding = self.relu(self.genres_fc(genres_vector))
        combined = summary_embedding + genres_embedding
        combined = self.output(combined)
        return combined / combined.norm(dim=1, keepdim=True)


In [98]:
# Model Başlatma ve Optimizasyon Ayarları
summary_dim = summary_tensor.shape[1]
genres_dim = genres_tensor.shape[1]
embedding_dim = 128

In [99]:
model = MovieRecommendationModel(summary_dim, genres_dim, embedding_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)
cosine_loss = nn.CosineEmbeddingLoss()

In [100]:
# Eğitim Fonksiyonu
def train(model, optimizer, epochs=10, batch_size=32):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for i in range(0, len(df), batch_size):
            batch_indices = np.random.choice(len(df), batch_size, replace=False)
            summary_batch = summary_tensor[batch_indices]
            genres_batch = genres_tensor[batch_indices]

            # Pozitif örnekler (kendisiyle benzerlik)
            output_pos = model(summary_batch, genres_batch)
            target_pos = torch.ones(batch_size)

            # Negatif örnekler (rastgele farklı film seçimi)
            neg_indices = np.random.choice(len(df), batch_size, replace=False)
            summary_neg = summary_tensor[neg_indices]
            genres_neg = genres_tensor[neg_indices]
            output_neg = model(summary_neg, genres_neg)
            target_neg = -torch.ones(batch_size)

            # Kayıp hesaplama (pozitif ve negatif örneklerle)
            loss_pos = cosine_loss(output_pos, output_pos, target_pos)
            loss_neg = cosine_loss(output_pos, output_neg, target_neg)
            loss = (loss_pos + loss_neg) / 2

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(df):.4f}")

In [101]:
# Modeli Eğit
train(model, optimizer, epochs=10, batch_size=32)

Epoch 1/10, Loss: 0.0015
Epoch 2/10, Loss: 0.0011
Epoch 3/10, Loss: 0.0010
Epoch 4/10, Loss: 0.0009
Epoch 5/10, Loss: 0.0009
Epoch 6/10, Loss: 0.0008
Epoch 7/10, Loss: 0.0008
Epoch 8/10, Loss: 0.0008
Epoch 9/10, Loss: 0.0008
Epoch 10/10, Loss: 0.0008


In [102]:
# Öneri Fonksiyonu
def recommend_movies(title, model, df, top_k=10):
    idx = df[df['title'] == title].index[0]
    summary_vector = summary_tensor[idx]
    genres_vector = genres_tensor[idx]

    with torch.no_grad():
        movie_embeddings = model(summary_tensor, genres_tensor)

    selected_movie_embedding = movie_embeddings[idx].unsqueeze(0)
    similarities = torch.mm(selected_movie_embedding, movie_embeddings.T).squeeze(0)
    _, top_indices = torch.topk(similarities, top_k + 1)

    recommended_movies = df.iloc[top_indices[1:].cpu().numpy()]['title']
    return recommended_movies

# Öneri örneği
movie_title = "Harry Potter and the Prisoner of Azkaban"
recommended_movies = recommend_movies(movie_title, model, df)
print("Önerilen Filmler:")
print(recommended_movies)

Önerilen Filmler:
9354          Harry Potter and the Chamber of Secrets
9925     Harry Potter and the Deathly Hallows: Part I
3711                                     Return to Oz
14838             Harry Potter and the Goblet of Fire
15623    Harry Potter and the Deathly Hallows: Part 2
2578                  Alice Through the Looking Glass
6527                              Alice in Wonderland
2735                                       Scooby-Doo
14193          Harry Potter and the Half-Blood Prince
12289       Harry Potter and the Order of the Phoenix
Name: title, dtype: object
